In [58]:
import pandas as pd
import json

In [57]:
file1_df = pd.read_json('raw_data/purchase_data.json')
file2_df = pd.read_json('raw_data/purchase_data2.json')
print (len(file1_df))
print (len(file2_df))
#pymoli_df = pd.merge(file1_df,file2_df, on="", how="inner")

780
78


In [66]:
file1_df.columns,file2_df.columns

(Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'),
 Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'))

In [67]:
pymoli_df = file1_df.append(file2_df)

In [73]:
playerCount = len(pymoli_df['SN'])
print (playerCount)

858
